In [1]:
import ee
import geemap
import geopandas as gpd
import numpy as np
from agroforestry.config import * 
from agroforestry.geeHelpers import *
from agroforestry.naipProcessing import *
from agroforestry.snicProcessing import *
from agroforestry.randomForest import *
from agroforestry.exportFunctions import *

In [2]:
#print
vsurfNoCor

['entropy_n',
 'nd_mean_neighborhood',
 'contrast_g',
 'contrast_n',
 'R',
 'G',
 'G_mean',
 'R_mean',
 'N',
 'savg_n_mean']

In [3]:
# establish connection with ee account. might require some additional configuration based on local machine 
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()# i

In [4]:
# type(grid24)
grid.Unique_ID
# grid24.Unique_ID
# gridSelect


0        X12-1
1        X12-2
2        X12-3
3        X12-4
4        X12-5
        ...   
768    X12-769
769    X12-770
770    X12-771
771    X12-772
772    X12-773
Name: Unique_ID, Length: 773, dtype: object

In [5]:
# define the aoi
aoiID = initGridID # something to itorate over for now is defined based on the input training dataset 
# or manually define it for where you want to apply the model too
# aoiID = "X12-601"
# # this becomes the AOI to used in the prepNAIP function. I'll need to edit it so that it converts the input data into a bbox 
# gridSelect = grid.loc[grid.Unique_ID == aoiID]


#Define bands to use
bandsToUse = vsurfNoCor

# select multiple grids level 1 
## want to pull thing from the csv rather than write it out. 
# aoiID = grid24
gridSelect =  grid.loc[grid.Unique_ID.isin(grid36.Unique_ID)].dissolve()

# after the desolve this gets assign a unique id some way. Might want to assign it the initGridID instead 
# might want to 
len(gridSelect.Unique_ID)

# len(grid8)



1

In [6]:
grid24

,Unnamed: 0,Unique_ID,poisition
0,1,X12-232,1
1,2,X12-233,1
2,3,X12-234,1
3,4,X12-280,1
4,5,X12-281,1
5,6,X12-282,1
6,7,X12-328,1
7,8,X12-329,1
8,9,X12-330,1
9,10,X12-183,2


In [7]:
# convert to a gee object 
aoi1 = geemap.gdf_to_ee(gridSelect)
# create a sub grid for downloading 
# downloadGrids = geemap.fishnet(aoi1.geometry(), rows=6, cols=4, delta=0)
aoiID

'X12-281'

In [8]:
Map = geemap.Map(center=(42.3, -98), zoom=10)
Map.add_basemap('HYBRID')
Map.addLayer(aoi1, {'color': '000000ff',
                    'width': 2,
                    'lineType': 'solid'},
             'area of interest')
# Map.addLayer(downloadGrids, {'color': '000000ff',
#                     'width': 2,
#                     'lineType': 'solid'},
#              'area of subgrid')

Map

Map(center=[42.3, -98], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(c…

In [9]:
# import training dataset 
# trainingData = gpd.read_file(filename="data/processed/trainingdataset_withClasses.geojson")
trainingData = gpd.read_file(filename="data/processed/" + str(initGridID) +"/"+ "agroforestrySamplingData_"+str(year)+".geojson") # initGridID defined int he config file

# divide the data into test train spilts 
trainingData = trainingData.sample(frac = 1)
# get rows 
total_rows = trainingData.shape[0]
# get train size 
train_size = int(total_rows*test_train_ratio)
 
# Split data into test and train
train = trainingData[0:train_size]
test = trainingData[train_size:]
# define the GEE objects
training = geemap.gdf_to_ee(gdf=train)
testing = geemap.gdf_to_ee(gdf=test)

# # print(type(trainingData))
# # select the training class of interest and drop unnecessary columns
# trainingSubset = trainingData
# # trainingSubset =  trainingData[trainingData.sampleStrat == "original"] ## will want to drop this as we wont have multiple sampling categories 
# # print(trainingSubset)
# # convert to ee object
# pointsEE = geemap.gdf_to_ee(gdf=trainingSubset)
# # subset testing and training data 
# training = pointsEE.filter(ee.Filter.gt('random', test_train_ratio))
# testing = pointsEE.filter(ee.Filter.lte('random',test_train_ratio))
# traing the rf model 
# rfCluster = trainRFModel(bands=bandsToUse_Cluster, inputFeature=training, nTrees=nTrees,setSeed=setSeed)
# rfPixel = trainRFModel(bands=bandsToUse_Pixel, inputFeature=training, nTrees=nTrees,setSeed=setSeed)
rfPixelTrim = trainRFModel(bands=bandsToUse,  inputFeature=training, nTrees=nTrees,setSeed=setSeed )
## run validation using the testing set 
# clusterValidation = testRFClassifier(classifier=rfCluster, testingData= testing)
# pixelValidation = testRFClassifier(classifier=rfPixel, testingData= testing)
pixelValidationTrim = testRFClassifier(classifier=rfPixelTrim, testingData= testing)


In [10]:
geePrint(training)


{'type': 'FeatureCollection', 'columns': {'B': 'Integer', 'B_mean': 'Float', 'G': 'Integer', 'G_mean': 'Float', 'N': 'Integer', 'N_mean': 'Float', 'R': 'Integer', 'R_mean': 'Number', 'contrast_g': 'Float', 'contrast_g_mean': 'Float', 'contrast_n': 'Float', 'contrast_n_mean': 'Float', 'entropy_g': 'Float', 'entropy_g_mean': 'Float', 'entropy_n': 'Float', 'entropy_n_mean': 'Float', 'gridID': 'String', 'id': 'String', 'nd': 'Float', 'nd_mean': 'Float', 'nd_mean_neighborhood': 'Float', 'nd_sd_neighborhood': 'Float', 'presence': 'Integer', 'random': 'Float', 'savg_g': 'Float', 'savg_g_mean': 'Float', 'savg_n': 'Float', 'savg_n_mean': 'Float', 'system:index': 'String'}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-97.0195373967035, 41.08097753063318]}, 'id': '0', 'properties': {'B': 51, 'B_mean': 87.85045623779297, 'G': 78, 'G_mean': 110.51373291015625, 'N': 167, 'N_mean': 133.50660705566406, 'R': 54, 'R_mean': 96.33672332763672, 'contrast_g': 455.742991727

In [11]:

# define export aoi
# exportAOI = ee.Feature(downloadGrids.toList(50).get(4))
# geePrint(exportAOI)
# exportAOI.geometry()
year


2020

In [12]:
### 
# Generate model based on year define in config 

# generate NAIP layer 
naipEE = prepNAIP(aoi=aoi1, year=year,windowSize=windowSize)
# geePrint(naipEE.bandNames())
# normal the naip data
# normalizedNAIP = normalize_by_maxes(img=naipEE, bandMaxes=bandMaxes)

# produce the SNIC object 
## filtering the image bands right away based on the single model output 
snicData = snicOutputs(naip = naipEE,
                       SNIC_SeedShape = SNIC_SeedShape, 
                       SNIC_SuperPixelSize = SNIC_SuperPixelSize, 
                       SNIC_Compactness = SNIC_Compactness, 
                       SNIC_Connectivity = SNIC_Connectivity,
                       # nativeScaleOfImage = nativeScaleOfImage, 
                       bandsToUse_Cluster = bandsToUse_Cluster).select(bandsToUse)
# apply the model and clip to aoi and reclass to unsigned 8bit image 
classifiedPixelsTrim = applyRFModel(imagery=snicData, bands=bandsToUse,classifier=rfPixelTrim).clip(aoi1).uint8()
# produce image for map 
demoImage = classifiedPixelsTrim #.clip(exportAOI)
# demoImage = classifiedPixelsTrim.clip(ee.Feature(downloadGrids.toList(50).get(10))).reproject(crs='EPSG:4326', scale=5)

geePrint(demoImage)


{'type': 'Image', 'bands': [{'id': 'classification', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [13]:
# print
# vsurfNoCor
# vsurfWithCor

In [14]:
# Set visualization parameters.
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['#f5f7f710', '#10c9a1'],
}
# add features to the map
Map.addLayer(demoImage, vis_params, str(year) + ' model')
# Map.addLayer(demoImage2010, vis_params,'2010 model',False)
# Map.addLayer(demoImage2020, vis_params,'2020 model',False)

Map

Map(center=[42.3, -98], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(c…

In [15]:
# export image to asset 
task = ee.batch.Export.image.toAsset(
  image = demoImage,
  description = str(initGridID) + "_" + str(year) + "_36grid",
  assetId = "projects/agroforestry2023/assets/"+ str(initGridID) + "_" + str(year) + "_36grid",
  region=aoi1.geometry(),
  scale=1,
  crs= demoImage.projection(),
  maxPixels = 1e13
)
task.start()

In [ ]:
str(initGridID)
str(year)

In [ ]:
# track the task 
import time
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

In [ ]:
geePrint(demoImage)